# Import Library

In [1]:
# Pandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, to_date, to_timestamp, when, expr, date_trunc, count, avg
from pyspark.sql.functions import explode, split, trim
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import count as Fcount

spark = SparkSession.builder \
    .appName("BNPL_Part1_Data") \
    .config("spark.driver.memory", "6g") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/11 14:31:48 WARN Utils: Your hostname, xuzhengs-MacBook-Pro-2.local, resolves to a loopback address: 127.0.0.1; using 10.13.6.57 instead (on interface en0)
25/09/11 14:31:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/11 14:31:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data Check - Part 2 & 3 & 4

In [2]:
p2_path = "part2_data/transaction2021A"
p2_df = spark.read.parquet(p2_path)

p3_path = "part3_data/transaction2021B"
p3_df = spark.read.parquet(p3_path)

p4_path = "part4_data/transaction2022A"
p4_df = spark.read.parquet(p4_path)

new_tbl_consumer = (
    spark.read
         .option("header", True)   
         .option("inferSchema", True)
         .csv("part1_data/new_tbl_consumer.csv")
)
consumer_details = spark.read.parquet("part1_data/consumer_user_details.parquet")

### 赵航宇数据
preprocessed_data = spark.read.parquet("preprocessed_data.parquet")


In [3]:
p2_df.show(10)
p2_df.count()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
|      3| 76819856970|  448.529684285612|5ace6a24-cdf0-4aa...|    2021-08-20|
|  18479| 67609108741|  86.4040605836911|d0e180f0-cb06-42a...|    2021-08-20|
|      3| 34096466752| 301.5793450525113|6fb1ff48-24bb-4f9...|    2021-08-20|
|  18482| 70501974849| 68.75486276223054|8505fb33-b69a-412...|    2021-08-20|
|      4| 49891706470| 48.89796461900801|ed11e477-b09f-4ae...|  

3643266

In [4]:
p3_df.show(10)
p3_df.count()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|
|      1| 46451548968| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|
|  14936| 89518629617|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|
|      1| 49167531725| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|
|  14936| 31101120643|25.228114942417797|8e301c0f-06ab-45c...|    2021-11-26|
|      2| 67978471888| 691.5028234458998|0380e9ad-b0e8-420...|    2021-11-26|
|  14936| 60956456424|102.13952056640888|5ac3da9c-5147-452...|    2021-11-26|
|      2| 47644196714| 644.5220654863093|4e368e44-86f8-4de...|    2021-11-26|
|  14938| 39649557865|209.12780951421405|4d78cd01-4bab-494...|    2021-11-26|
|      3| 88402174457| 141.0387993699113|c50c957d-ecfc-430...|  

4508106

In [5]:
p4_df.show(10)
p4_df.count() 

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  11139| 96152467973|16.213590228273233|785b0080-9e4b-471...|    2022-08-20|
|      1| 98973094975| 86.97955945703498|2560f7b0-ee5d-4b3...|    2022-08-20|
|  11139| 56762458844|31.513502323509194|0311717b-8b5b-410...|    2022-08-20|
|      1| 89502033586|124.18468694868491|f8891626-f098-45b...|    2022-08-20|
|  11139| 96161808980|61.620445567668966|d90a421f-f1da-4bf...|    2022-08-20|
|      2| 72472909171| 32.26524985312485|523e0403-b677-450...|    2022-08-20|
|  11139| 91923722701|11.331586767322225|f45a842b-0366-41d...|    2022-08-20|
|      3| 46380096952|119.80011239189334|58d0f423-037c-43f...|    2022-08-20|
|  11140| 79283124876|198.13027742225435|60b12d41-41d6-4c1...|    2022-08-20|
|      4| 67202032418|206.20865323560022|64a05a23-a078-481...|  

6044133

In [6]:
new_tbl_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows


In [7]:
preprocessed_data.show(10)

+-------+------------+------------------+--------------------+
|user_id|merchant_abn|      dollar_value|            order_id|
+-------+------------+------------------+--------------------+
|      1| 28000487688|133.22689421562643|0c37b3f7-c7f1-48c...|
|  18485| 62191208634| 79.13140006851712|9e18b913-0465-4fd...|
|      1| 83690644458|30.441348317517228|40a2ff69-ea34-465...|
|  18488| 39649557865| 962.8133405407585|f4c1a5ae-5b76-40d...|
|      2| 80779820715| 48.12397733548124|cd09bdd6-f56d-489...|
|  18489| 43186523025| 98.14878546968934|9008a98e-1b02-4de...|
|      3| 29566626791| 46.33087226118639|26b7574e-81c2-455...|
|  18490| 93558142492|232.83335268750145|2bda0665-796f-4f2...|
|      3| 32361057556| 87.34942171371054|633a7656-2fcc-4b8...|
|  18491| 64974914166|130.12601873970038|4bc15338-83eb-43d...|
+-------+------------+------------------+--------------------+
only showing top 10 rows


# Final Data Merge

In [8]:
# 先把 user_id -> consumer_id 的映射补充进交易表
df_with_consumer = (
    preprocessed_data
    .join(consumer_details, on="user_id", how="left")
)

# Step 2: 再用 consumer_id 去 join consumer 画像表
df_enriched = (
    df_with_consumer
    .join(new_tbl_consumer, on="consumer_id", how="left")
)

# Step 3: 选择你想要的字段
df_enriched_selected = df_enriched.select(
    "user_id",
    "consumer_id",
    "merchant_abn",
    "order_id",
    "dollar_value",
    "state",
    "postcode",
    "gender"
)

df_enriched_selected.show(30, truncate=False)

+-------+-----------+------------+------------------------------------+------------------+-----+--------+-----------+
|user_id|consumer_id|merchant_abn|order_id                            |dollar_value      |state|postcode|gender     |
+-------+-----------+------------+------------------------------------+------------------+-----+--------+-----------+
|2883   |75264      |94493496784 |1b237258-3f9a-4058-8e5e-8651c628cd35|86.12019907563348 |VIC  |3123    |Female     |
|18519  |147657     |58741105428 |7f623ae1-5373-4a26-926d-8df335c2bec5|43.31490194582448 |NSW  |2218    |Male       |
|10957  |149874     |35909341340 |128cabaa-187b-4f4d-8473-f2dc581fb40f|181.78257205494356|VIC  |3483    |Male       |
|16800  |186459     |76767266140 |ac94d659-5c1e-445b-b9ea-ba029dc982d6|124.5343717624213 |WA   |6964    |Undisclosed|
|16800  |186459     |91923722701 |e084f4c9-8829-4ea6-9a98-8050a556a520|5.971913169283436 |WA   |6964    |Undisclosed|
|8046   |279655     |61242800720 |a74ec34f-40c7-437a-ba8